In [1]:
import ee
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import geemap 

In [2]:
def processing_grid(aoi, grid_size):

    boundbox = (
        aoi.geometry().bounds().buffer(distance=1, proj=ee.Projection("EPSG:4326"))
    )

    # return the list of coordinates
    list_cords = ee.Array.cat(boundbox.coordinates(), 1)

    # get the X and Y -coordinates
    x_cords = list_cords.slice(1, 0, 1)
    y_cords = list_cords.slice(1, 1, 2)

    # reduce the arrays to find the max (or min) value
    x_min = x_cords.reduce("min", [0]).get([0, 0])
    x_max = x_cords.reduce("max", [0]).get([0, 0])
    y_min = y_cords.reduce("min", [0]).get([0, 0])
    y_max = y_cords.reduce("max", [0]).get([0, 0])

    xx = ee.List.sequence(
        x_min, ee.Number(x_max).subtract(ee.Number(grid_size).multiply(0.9)), grid_size
    )
    yy = ee.List.sequence(
        y_min, ee.Number(y_max).subtract(ee.Number(grid_size).multiply(0.9)), grid_size
    )

    def map_over_x(_x):
        def map_over_y(_y):
            x1 = ee.Number(_x)
            x2 = ee.Number(_x).add(ee.Number(grid_size))
            y1 = ee.Number(_y)
            y2 = ee.Number(_y).add(ee.Number(grid_size))

            cords = ee.List([x1, y1, x2, y2])
            rect = ee.Algorithms.GeometryConstructors.Rectangle(
                cords, "EPSG:4326", False
            )
            return ee.Feature(rect)

        return yy.map(map_over_y)

    cells = xx.map(map_over_x).flatten()

    return ee.FeatureCollection(cells).filterBounds(aoi)

In [ ]:
aoi = ee.FeatureCollection("USDOS/LSIB/2017").filter(ee.Filter.Or(
    ee.Filter.stringContains('COUNTRY_NA', 'Congo')
    #ee.Filter.stringContains('COUNTRY_NA', 'Equatorial Guinea'),
    #ee.Filter.stringContains('COUNTRY_NA', 'Gabon'),
    #ee.Filter.stringContains('COUNTRY_NA', 'Central African'),
    #ee.Filter.stringContains('COUNTRY_NA', 'Cameroon'),
)).union(100)

In [3]:
from sampling_handler.misc import ee_helpers

In [ ]:
%%time
points_fc = ee.FeatureCollection('users/andreasvollrath/COGSamples').sort('point_id').limit(25000)
gmt = '121212'
_, points_fc = ee_helpers._ee_export_table(
        ee_fc=points_fc,
        description=f"tmp_esbae_points_{gmt}",
        asset_id=f"tmp_esbae_points_{gmt}",
        sub_folder=f"tmp_esbae_{gmt}"
    )
points_fc = ee.FeatureCollection(points_fc)
aoi = ee.FeatureCollection(points_fc.geometry().convexHull(100))


#_, aoi = ee_helpers._ee_export_table(
#        ee_fc=aoi,
#        description=f"tmp_esbae_aoi_{gmt}",
#        asset_id=f"tmp_esbae_aoi_{gmt}",
#        sub_folder=f"tmp_esbae_{gmt}"
#    )
aoi = ee.FeatureCollection('users/andreasvollrath/tmp_esbae_121212/tmp_esbae_aoi_121212')
#chunks_fc = aoi.coveringGrid(aoi.projection(), 5000)

chunks_fc = processing_grid(aoi, 0.05)

to_proc = chunks_fc.map(
        lambda x: ee.Feature(x).set(
            'intersects', points_fc.filterBounds(x.geometry()).size()
        )
    )
to_proc = to_proc.filter(ee.Filter.And(
        ee.Filter.gt('intersects', 0),
        ee.Filter.lt('intersects', 500)
    ))

_, to_proc = ee_helpers._ee_export_table(
        ee_fc=to_proc,
        description=f"tmp_esbae_grid_{gmt}",
        asset_id=f"tmp_esbae_grid_{gmt}",
        sub_folder=f"tmp_esbae_{gmt}"
    )
to_proc = ee.FeatureCollection(to_proc)
processing_chunks = ee.FeatureCollection(to_proc).aggregate_array('.geo').getInfo()

print(processing_chunks) #.toList(processing_chunks.size()).getInfo())

In [ ]:
{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[10.652077943476744, -4.459033997179562], [11.152077943476744, -4.459033997179562], [11.152077943476744, -3.959033997179562], [10.652077943476744, -3.959033997179562], [10.652077943476744, -4.459033997179562]]]}, 'id': '61', 'properties': {'intersects': 0}}

In [ ]:
Map = geemap.Map(center=(0, 10), zoom=4)
Map.addLayer(points_fc)
Map.addLayer(chunks_fc)
Map

In [ ]:
grid = aoi.coveringGrid(aoi.projection(), spacing).filterBounds(aoi)
    
    